In [1]:
import pandas as pd
import zipfile

pd.options.display.min_rows = 100

In [3]:
# extract txt, file extension is erroneously gz, but its actually a zip file :)
fname = "ATH_GO_GOSLIM.txt"
fname_gz = f"{fname}.gz"

with zipfile.ZipFile(fname_gz, 'r') as z:
    z.extractall(".")

In [4]:
# from ATH_GO.README.txt
columns = [
    "locus_name",
    "tair_acc",
    "obj_name",
    "rel_type",
    "go_term",
    "go_id",
    "tair_id",
    "aspect",
    "go_slim",
    "evidence_code",
    "evidence_desc",
    "evidence_with",
    "reference",
    "annotator",
    "date"
]


# read into dataframe
df0 = pd.read_csv(fname, sep="\t", names=columns, skiprows=[0,1,2,3], index_col=False, header=0)
df0.head()
    


,locus_name,tair_acc,obj_name,rel_type,go_term,go_id,tair_id,aspect,go_slim,evidence_code,evidence_desc,evidence_with,reference,annotator,date
0,AT1G01010,locus:2200935,AT1G01010,acts upstream of or within,response to oxidative stress,GO:0006979,6625,P,response to stress,IEA,traceable computational prediction,AGI_LocusCode:AT5G19875,Publication:501796011|PMID:34562334,klaasvdp,2022-11-14
1,AT1G01010,locus:2200935,AT1G01010,acts upstream of or within,response to abscisic acid,GO:0009737,11395,P,response to chemical,IEA,traceable computational prediction,AGI_LocusCode:AT4G27410,Publication:501796011|PMID:34562334,klaasvdp,2022-11-14
2,AT1G01010,locus:2200935,AT1G01010,acts upstream of or within,response to lipid,GO:0033993,28865,P,response to chemical,IEA,traceable computational prediction,AGI_LocusCode:AT4G27410|AGI_LocusCode:AT2G0299...,Publication:501796011|PMID:34562334,klaasvdp,2022-11-14
3,AT1G01010,locus:2200935,AT1G01010,acts upstream of or within,oxoacid metabolic process,GO:0043436,21524,P,other cellular processes,IEA,traceable computational prediction,AGI_LocusCode:AT5G63790,Publication:501796011|PMID:34562334,klaasvdp,2022-11-14
4,AT1G01010,locus:2200935,AT1G01010,acts upstream of or within,defense response to other organism,GO:0098542,46569,P,response to external stimulus,IEA,traceable computational prediction,AGI_LocusCode:AT2G43510|AGI_LocusCode:AT4G1473...,Publication:501796011|PMID:34562334,klaasvdp,2022-11-14


In [5]:
select_columns = ["locus_name", "go_term"]
genes = [
    "AT1G49570",
    "AT3G00400",
    "AT2G14610",
    "AT2G44240",
    "AT3G28510",
    "AT1G28590",
    "AT1G14880",
    "AT5G58180",
    "AT1G28005",
    "AT3G22238",
    "AT3G22234",
    "AT1G65483",
    "AT4G10860",
    "AT2G29250",
    "AT4G05250"
]

df = df0[select_columns] # select gene name and function
df = df.drop_duplicates() # drop duplicates
df = df.query("locus_name in @genes") # select the genes

df

,locus_name,go_term
26632,AT1G14880,defense response to fungus
26633,AT1G14880,response to wounding
26635,AT1G14880,response to salicylic acid
26636,AT1G14880,defense response to bacterium
26637,AT1G14880,mRNA binding
26638,AT1G14880,regulation of defense response
26639,AT1G14880,plant-type vacuole
26642,AT1G14880,response to inorganic substance
26644,AT1G14880,extracellular region
26645,AT1G14880,plant organ senescence


In [6]:
dfg = df.groupby("go_term", as_index=False).apply(lambda x: x)
dfg

locus_name                                            go_term
0  427399  AT5G58180                                    Golgi apparatus
1  427398  AT5G58180                             SNAP receptor activity
2  427395  AT5G58180                                      SNARE complex
3  127262  AT2G14610                      activation of immune response
4  48046   AT1G28005                                 biological_process
   236361  AT3G28510                                 biological_process
   287775  AT4G10860                                 biological_process
5  65692   AT1G49570                                          cell wall
6  287774  AT4G10860                                 cellular_component
7  150849  AT2G29250                                        chloroplast
8  127261  AT2G14610                                  cytokine activity
9  127256  AT2G14610                                cytokine production
10 26647   AT1G14880                                          cytoplasm
   89176   AT1G65483                                          cytoplasm
11 283583  AT4G05250                                            cytosol
12 49078   AT1G28590                                   defense response
   127267  AT2G14610                                   defense response
13 26636   AT1G14880                      defense response to bacterium
   89172   AT1G65483                      defense response to bacterium
   150843  AT2G29250                      defense response to bacterium
14 26632   AT1G14880                         defense response to fungus
   89173   AT1G65483                         defense response to fungus
15 150848  AT2G29250                      defense response to oomycetes
16 236359  AT3G28510                              endoplasmic reticulum
17 427403  AT5G58180  endoplasmic reticulum to Golgi vesicle-mediate...
18 26644   AT1G14880                               extracellular region
   49075   AT1G28590                               extracellular region
   65693   AT1G49570                               extracellular region
   127252  AT2G14610                               extracellular region
   178678  AT2G44240                               extracellular region
...              ...                                                ...
28 283588  AT4G05250                                            nucleus
29 49079   AT1G28590          organonitrogen compound catabolic process
30 26645   AT1G14880                             plant organ senescence
31 127255  AT2G14610                               plant-type cell wall
32 26639   AT1G14880                                 plant-type vacuole
33 150844  AT2G29250                                    plasma membrane
   236358  AT3G28510                                    plasma membrane
34 283577  AT4G05250  polyubiquitin modification-dependent protein b...
35 283579  AT4G05250                                 proteasome binding
36 283578  AT4G05250  proteasome-mediated ubiquitin-dependent protei...
37 127264  AT2G14610                                    protein binding
   427402  AT5G58180                                    protein binding
38 150850  AT2G29250                            protein kinase activity
39 26638   AT1G14880                     regulation of defense response
40 26642   AT1G14880                    response to inorganic substance
41 49076   AT1G28590                          response to jasmonic acid
42 65695   AT1G49570                       response to oxidative stress
   178680  AT2G44240                       response to oxidative stress
43 26635   AT1G14880                         response to salicylic acid
   89178   AT1G65483                         response to salicylic acid
44 127258  AT2G14610                             response to vitamin B1
45 127253  AT2G14610                      response to water deprivation
46 26633   AT1G14880                               response to wounding
   65694   AT1G49570                              

In [7]:
# query by go_term
list(dfg[dfg["go_term"] == "defense response"]["locus_name"])

['AT1G28590', 'AT2G14610']